First I will mount my google drive to acces the csv files from one folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


I am going to import the data sources from the City of Tucson and merge them into one data frame using pandas.

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from scipy import stats

# set theme for seaborn
sns.set_theme(style = 'whitegrid')


# set settings to default='warn' to avoid stopping execution of code
pd.options.mode.chained_assignment = None 

# grab datasets from website link
race_df = pd.read_csv('/content/gdrive/My Drive/Colab Datasets/Neighborhood_Race_Demographics.csv')
education_df = pd.read_csv('/content/gdrive/My Drive/Colab Datasets/Neighborhood_Educational_Attainment.csv')


# merge both dataframes together
merged_df = race_df.merge(education_df)
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 73 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   OBJECTID                       159 non-null    int64  
 1   NAME                           159 non-null    object 
 2   WARD                           159 non-null    int64  
 3   DATASOURCE                     159 non-null    object 
 4   ID                             159 non-null    int64  
 5   sourceCountry                  159 non-null    object 
 6   ENRICH_FID                     159 non-null    int64  
 7   aggregationMethod              159 non-null    object 
 8   populationToPolygonSizeRating  159 non-null    float64
 9   apportionmentConfidence        159 non-null    float64
 10  HasData                        159 non-null    int64  
 11  WHITE_CY                       159 non-null    int64  
 12  BLACK_CY                       159 non-null    int

I am then going to rearrange the columns and pick only those that I am interested in reviewing.

In [ ]:
# create smaller cleaned up dataframe with only columns of interest (neighborhood info, race, education)
columns = list(merged_df.columns.values)
demog_df = merged_df[columns[1:3] + columns[11:12] + columns[18:19] + columns[12:17] + columns[64:72]]

demog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NAME        159 non-null    object
 1   WARD        159 non-null    int64 
 2   WHITE_CY    159 non-null    int64 
 3   HISPPOP_CY  159 non-null    int64 
 4   BLACK_CY    159 non-null    int64 
 5   AMERIND_CY  159 non-null    int64 
 6   ASIAN_CY    159 non-null    int64 
 7   PACIFIC_CY  159 non-null    int64 
 8   OTHRACE_CY  159 non-null    int64 
 9   NOHS_CY     159 non-null    int64 
 10  SOMEHS_CY   159 non-null    int64 
 11  HSGRAD_CY   159 non-null    int64 
 12  GED_CY      159 non-null    int64 
 13  SMCOLL_CY   159 non-null    int64 
 14  ASSCDEG_CY  159 non-null    int64 
 15  BACHDEG_CY  159 non-null    int64 
 16  GRADDEG_CY  159 non-null    int64 
dtypes: int64(16), object(1)
memory usage: 22.4+ KB


Add additional columns:
Totals for population count, education response count
Combined totals for none and some HS; HS and GED; some, Associates, Bachelors, and Grad college

In [ ]:
# add all races for total population
demog_df['total_pop'] = demog_df['WHITE_CY'] + demog_df['BLACK_CY']\
 + demog_df['AMERIND_CY'] + demog_df['ASIAN_CY'] + demog_df['PACIFIC_CY'] \
 + demog_df['OTHRACE_CY'] + demog_df['HISPPOP_CY']

# add all responses for total education attendees
demog_df['total_ed'] = demog_df['ASSCDEG_CY'] + demog_df['BACHDEG_CY']\
 + demog_df['GED_CY'] + demog_df['GRADDEG_CY'] + demog_df['HSGRAD_CY']\
  + demog_df['NOHS_CY'] + demog_df['SMCOLL_CY'] + demog_df['SOMEHS_CY']

# add all non HS graduates
demog_df['less_HS'] = demog_df['NOHS_CY'] + demog_df['SOMEHS_CY']

# add all HS/GED
demog_df['HS_GED'] = demog_df['HSGRAD_CY'] + demog_df['GED_CY']

# add all post HS education
demog_df['post_HS'] = demog_df['GRADDEG_CY'] + demog_df['SMCOLL_CY'] \
+ demog_df['ASSCDEG_CY'] + demog_df['BACHDEG_CY']

Create second dataframe that is grouped by Wards for average comparison

In [ ]:
# group by wards and reset index count
ward_df = demog_df.groupby('WARD').sum()

# test dataframe
ward_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 1 to 6
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   WHITE_CY    6 non-null      int64
 1   HISPPOP_CY  6 non-null      int64
 2   BLACK_CY    6 non-null      int64
 3   AMERIND_CY  6 non-null      int64
 4   ASIAN_CY    6 non-null      int64
 5   PACIFIC_CY  6 non-null      int64
 6   OTHRACE_CY  6 non-null      int64
 7   NOHS_CY     6 non-null      int64
 8   SOMEHS_CY   6 non-null      int64
 9   HSGRAD_CY   6 non-null      int64
 10  GED_CY      6 non-null      int64
 11  SMCOLL_CY   6 non-null      int64
 12  ASSCDEG_CY  6 non-null      int64
 13  BACHDEG_CY  6 non-null      int64
 14  GRADDEG_CY  6 non-null      int64
 15  total_pop   6 non-null      int64
 16  total_ed    6 non-null      int64
 17  less_HS     6 non-null      int64
 18  HS_GED      6 non-null      int64
 19  post_HS     6 non-null      int64
dtypes: int64(20)
memory usage: 1008.0 by

Add columns that show the overall percentage of ethnicity from total population per neighborhood, and overall education percentage from total education responses per neighborhood

In [ ]:
# percentage white population
demog_df['white_%'] = (demog_df['WHITE_CY'] / demog_df['total_pop'])*100

# percentage hispanic population per neighborhood
demog_df['hisp_%'] = (demog_df['HISPPOP_CY'] / demog_df['total_pop'])*100

# percentage black population per neighborhood
demog_df['black_%'] = (demog_df['BLACK_CY'] / demog_df['total_pop'])*100

# percentage Tribal population per neighborhood
demog_df['tribal_%'] = (demog_df['AMERIND_CY'] / demog_df['total_pop'])*100

# percentage other population per neighborhood
demog_df['other_%'] = ((demog_df['OTHRACE_CY'] + demog_df['ASIAN_CY']\
                           + demog_df['PACIFIC_CY'])/ demog_df['total_pop'])*100


# create columns for percentage of education completion out of total population of ward
# and append to demog_df dataframe
demog_df['less_HS _%'] = demog_df['less_HS'] / demog_df['total_ed'] * 100
demog_df['HS _%'] = demog_df['HS_GED'] / demog_df['total_ed'] * 100
demog_df['post_HS _%'] = demog_df['post_HS'] / demog_df['total_ed'] * 100



# view clean dataframe
demog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NAME        159 non-null    object 
 1   WARD        159 non-null    int64  
 2   WHITE_CY    159 non-null    int64  
 3   HISPPOP_CY  159 non-null    int64  
 4   BLACK_CY    159 non-null    int64  
 5   AMERIND_CY  159 non-null    int64  
 6   ASIAN_CY    159 non-null    int64  
 7   PACIFIC_CY  159 non-null    int64  
 8   OTHRACE_CY  159 non-null    int64  
 9   NOHS_CY     159 non-null    int64  
 10  SOMEHS_CY   159 non-null    int64  
 11  HSGRAD_CY   159 non-null    int64  
 12  GED_CY      159 non-null    int64  
 13  SMCOLL_CY   159 non-null    int64  
 14  ASSCDEG_CY  159 non-null    int64  
 15  BACHDEG_CY  159 non-null    int64  
 16  GRADDEG_CY  159 non-null    int64  
 17  total_pop   159 non-null    int64  
 18  total_ed    159 non-null    int64  
 19  less_HS     159 non-null    i

Add columns that show the overall percentage of ethnicity from total population per ward, and overall education percentage from total education responses per ward

In [ ]:
# percentage white population
ward_df['white_%'] = (ward_df['WHITE_CY'] / ward_df['total_pop'])*100

# percentage hispanic population per neighborhood
ward_df['hisp_%'] = (ward_df['HISPPOP_CY'] / ward_df['total_pop'])*100

# percentage black population per neighborhood
ward_df['black_%'] = (ward_df['BLACK_CY'] / ward_df['total_pop'])*100

# percentage Tribal population per neighborhood
ward_df['tribal_%'] = (ward_df['AMERIND_CY'] / ward_df['total_pop'])*100

# percentage other population per neighborhood
ward_df['other_%'] = ((ward_df['OTHRACE_CY'] + ward_df['ASIAN_CY']\
                           + ward_df['PACIFIC_CY'])/ ward_df['total_pop'])*100


# create columns for percentage of education completion out of total population of ward
# and append to ward_df dataframe
ward_df['less_HS _%'] = ward_df['less_HS'] / ward_df['total_ed'] * 100
ward_df['HS _%'] = ward_df['HS_GED'] / ward_df['total_ed'] * 100
ward_df['post_HS _%'] = ward_df['post_HS'] / ward_df['total_ed'] * 100


# view ward_df
ward_df.head()



,WHITE_CY,HISPPOP_CY,BLACK_CY,AMERIND_CY,ASIAN_CY,PACIFIC_CY,OTHRACE_CY,NOHS_CY,SOMEHS_CY,HSGRAD_CY,GED_CY,SMCOLL_CY,ASSCDEG_CY,BACHDEG_CY,GRADDEG_CY,total_pop,total_ed,less_HS,HS_GED,post_HS,white_%,hisp_%,black_%,tribal_%,other_%,less_HS _%,HS _%,post_HS _%
WARD,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,47204,62685,2622,3948,1138,107,24730,5306,7327,11697,3183,12441,3746,5071,2857,142434,51628,12633,14880,24115,33.140964,44.009857,1.840853,2.771810,18.236517,24.469280,28.821570,46.709150
2,34522,9832,2173,495,1604,102,2908,632,1348,5437,964,8166,3513,7224,5092,51636,32376,1980,6401,23995,66.856457,19.040979,4.208304,0.958634,8.935626,6.115641,19.770818,74.113541
3,42168,26779,4192,2561,3163,222,10339,2439,3958,6491,1791,10549,3110,7933,5218,89424,41489,6397,8282,26810,47.155126,29.946099,4.687780,2.863884,15.347110,15.418545,19.961918,64.619538
4,50413,20999,4516,896,2810,236,6535,965,2786,9200,2168,13303,5793,7949,4077,86405,46241,3751,11368,31122,58.345003,24.302992,5.226549,1.036977,11.088479,8.111849,24.584243,67.303908
5,35117,49855,3572,2503,1121,81,20427,4896,5713,9736,2284,9355,2562,3389,1839,112676,39774,10609,12020,17145,31.166353,44.246335,3.170152,2.221414,19.195747,26.673204,30.220747,43.106049


Start exploring the descriptive statistics of dataframes for ward - for accurate count of averages. demmog_df does not describe the percentages accurately

In [ ]:

ward_df.describe()

,WHITE_CY,HISPPOP_CY,BLACK_CY,AMERIND_CY,ASIAN_CY,PACIFIC_CY,OTHRACE_CY,NOHS_CY,SOMEHS_CY,HSGRAD_CY,GED_CY,SMCOLL_CY,ASSCDEG_CY,BACHDEG_CY,GRADDEG_CY,total_pop,total_ed,less_HS,HS_GED,post_HS,white_%,hisp_%,black_%,tribal_%,other_%,less_HS _%,HS _%,post_HS _%
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,42923.666667,31314.833333,3312.166667,1961.833333,2143.666667,150.500000,11766.500000,2584.500000,3886.833333,8074.833333,2009.500000,10531.000000,3682.333333,6926.166667,4798.666667,93573.166667,42493.833333,6471.333333,10084.333333,25938.166667,49.612967,30.673236,3.780253,1.931225,14.002320,14.791029,23.457516,61.751454
std,6833.674639,20496.978884,930.508768,1283.507603,960.208866,65.601067,8811.643496,2046.281579,2267.811405,2505.773287,739.688245,1982.518096,1110.666947,2344.994705,2735.827382,30970.899260,6468.011052,4283.559579,3212.659376,5556.108041,14.974732,11.003875,1.207154,0.830049,4.211058,8.964565,5.262329,13.655330
min,34522.000000,9832.000000,2173.000000,495.000000,1121.000000,81.000000,2908.000000,632.000000,1348.000000,5437.000000,964.000000,8166.000000,2562.000000,3389.000000,1839.000000,51636.000000,32376.000000,1980.000000,6401.000000,17145.000000,31.166353,19.040979,1.840853,0.958634,8.935626,6.115641,17.385801,43.106049
25%,36879.750000,18554.000000,2666.000000,1014.000000,1254.500000,103.250000,5878.750000,1041.000000,2338.250000,6038.750000,1698.000000,9359.250000,3175.000000,5609.250000,3162.000000,80749.250000,40202.750000,3531.250000,7736.750000,24025.000000,36.644504,22.945613,3.264584,1.211391,11.118969,7.996205,19.818593,51.186747
50%,44686.000000,23889.000000,3185.000000,1935.500000,2207.000000,131.000000,8437.000000,1854.000000,3372.000000,7845.500000,1979.500000,9960.500000,3441.500000,7578.500000,4584.500000,87914.500000,42472.000000,5074.000000,9825.000000,25462.500000,52.750065,27.124546,3.878092,1.978023,13.278774,11.765197,22.273081,65.961723
75%,47889.500000,44086.000000,4037.000000,2546.500000,2972.000000,205.250000,17905.000000,4281.750000,5274.250000,9602.000000,2255.000000,11968.000000,3687.750000,7945.000000,5186.500000,106863.000000,45544.500000,9556.000000,11857.000000,30044.000000,60.346674,40.493918,4.567911,2.634211,17.514165,22.206596,27.762238,72.411133
max,50413.000000,62685.000000,4516.000000,3948.000000,3163.000000,236.000000,24730.000000,5306.000000,7327.000000,11697.000000,3183.000000,13303.000000,5793.000000,9991.000000,9709.000000,142434.000000,51628.000000,12633.000000,14880.000000,32442.000000,66.856457,44.246335,5.226549,2.863884,19.195747,26.673204,30.220747,74.656541


From the descriptive statistics, you can see that the mean for undereducation (HS/ged completion) is 23.4. This is the value I will use to split the demog_df into higher and lower educated

In [ ]:
# create dataframes for hypothesis that 
neighborhood_educated = demog_df.iloc[(demog_df['HS _%'] > 23.4).values]
neighborhood_less_HS = demog_df.iloc[(demog_df['HS _%'] < 23.4).values]


With the demog_df split into more and less educated dataframes, I can do a t-test to see if there is a correlation between education and wards and education and ethnicity.

In [ ]:
# initalize column counter for first column of ethnicity
col_num = 2

# iterate through columns 2-8
while col_num < 9:
  # get name of column
  col_name = demog_df.columns[col_num]
  
  # get st. dev. and p value for column of both dataframes
  tw, pw = stats.ttest_ind(neighborhood_educated[col_name], neighborhood_less_HS[col_name])
  
  # increase counter
  col_num += 1

  print("The standard deviation for {} is {} with a p-value of {} ".format(col_name, tw, pw))

The standard deviation for WHITE_CY is 1.839288031013867 with a p-value of 0.06777381945362408 
The standard deviation for HISPPOP_CY is 5.256340382856412 with a p-value of 4.763253916875568e-07 
The standard deviation for BLACK_CY is 2.8292944057364675 with a p-value of 0.00527863655331421 
The standard deviation for AMERIND_CY is 5.178505019400797 with a p-value of 6.810521871352755e-07 
The standard deviation for ASIAN_CY is -1.058859703586984 with a p-value of 0.2913004514425794 
The standard deviation for PACIFIC_CY is 0.8658814207043 with a p-value of 0.3878849050956896 
The standard deviation for OTHRACE_CY is 5.649494293949966 with a p-value of 7.439111698916372e-08 


p-value analysis summary


From the descriptive statistics, you can see that the mean white population percentage per ward is 49.6. This will be the value I use to split the demog_df dataframe into white majority vs white minority for my hypothesis based on ethnicity.

In [ ]:
# create dataframes for hypothesis that race makes a difference in education completion
wards_white_maj = demog_df.iloc[(demog_df['white_%'] > 49.6).values]
wards_white_min = demog_df.iloc[(demog_df['white_%'] < 49.6).values]

In [ ]:
# initalize column counter at first education column
col_nums = 19

# iterate through columns 19-21
while col_nums < 22:
  # get name of column
  col_name = demog_df.columns[col_nums]
  
  # get st. dev. and p value for column of both dataframes
  tw, pw = stats.ttest_ind(wards_white_maj[col_name], wards_white_min[col_name])
  
  # increase counter
  col_nums += 1

  print("The standard deviation for {} is {} with a p-value of {} ".format(col_name, tw, pw))

The standard deviation for less_HS is -5.56850376515166 with a p-value of 1.0979286853436549e-07 
The standard deviation for HS_GED is -2.869558951742566 with a p-value of 0.004680746169284997 
The standard deviation for post_HS is 1.5043265747986083 with a p-value of 0.13451919975775684 
